# Assignment

## Step 1

### We start by importing the necessary libraries

In [3]:
pip install lxml

     |████████████████████████████████| 5.8MB 23.4MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install geocoder

     |████████████████████████████████| 102kB 5.0MB/s ta 0:00:011
     |████████████████████████████████| 92kB 7.0MB/s eta 0:00:011
     |████████████████████████████████| 829kB 7.9MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
Note: you may need to restart the kernel to use updated packages.


### Instead of using beautiful soup, we use pandas to import the table from Wikipedia using the URL

In [6]:
import pandas as pd
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

df=pd.read_html(url, header=0)[0]

### Filtering out all Boroughs with 'Not Assigned' values

In [7]:
df1=df[df['Borough']!='Not assigned']
df1.reset_index(inplace=True)

In [8]:
df1=df1.drop(['index'],axis=1)
df1.head(10)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [9]:
df1.shape

(103, 3)

### Importing Location Data

In [10]:
!wget -q -O 'canada_data.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [11]:
CanadaPC=pd.read_csv('canada_data.csv')

In [12]:
df2=pd.merge(df1,CanadaPC,left_on='Postal Code',right_on='Postal Code',how='inner')

In [13]:
df2

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing Centre,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [14]:
pip install geopy

     |████████████████████████████████| 122kB 6.1MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [15]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Obtaining Location data for Toronto

In [16]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="tr_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### Creating a map with markers for all neighborhoods

In [17]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Obtaining Data for one Neighborhood from Foursquare API

In [18]:
CLIENT_ID = 'GSIXGNCSRHTN5X4WWDUJREPKO4YPINV0YGAUHSDCGYACMHQV' # your Foursquare ID
CLIENT_SECRET = 'UM10BB0EI5LFX0OXA2KTDPQYD22QTKENB3PO4GCTVIYFCRSM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GSIXGNCSRHTN5X4WWDUJREPKO4YPINV0YGAUHSDCGYACMHQV
CLIENT_SECRET:UM10BB0EI5LFX0OXA2KTDPQYD22QTKENB3PO4GCTVIYFCRSM


In [19]:
df2.loc[0, 'Neighborhood']

'Parkwoods'

In [20]:
neighborhood_latitude = df2.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df2.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df2.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


In [21]:
LIMIT=100
radius=500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [26]:
result1 = requests.get(url).json()
result1

{'meta': {'code': 200, 'requestId': '5ebc15fd211536001bbb84e6'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': 

### Structuring Data obtained from API call

In [23]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [27]:
venues = result1['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [28]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


### Defining Function to run API call for all Neighborhoods and structuring data

In [29]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [30]:
toronto_venues = getNearbyVenues(names=df2['Neighborhood'],
                                   latitudes=df2['Latitude'],
                                   longitudes=df2['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview
The Danforth West, Ri

In [67]:
print(toronto_venues.shape)
toronto_venues['Venue Category'].unique()

(2119, 7)


array(['Park', 'Food & Drink Shop', 'Hockey Arena', 'Coffee Shop',
       'Portuguese Restaurant', 'French Restaurant', 'Pizza Place',
       'Bakery', 'Breakfast Spot', 'Distribution Center', 'Spa',
       'Restaurant', 'Gym / Fitness Center', 'Historic Site',
       'Farmers Market', 'Chocolate Shop', 'Pub', 'Performing Arts Venue',
       'Dessert Shop', 'Yoga Studio', 'Café', 'Theater', 'Event Space',
       'Ice Cream Shop', 'Shoe Store', 'Mexican Restaurant',
       'Art Gallery', 'Asian Restaurant', 'Cosmetics Shop', 'Bank',
       'Electronics Store', 'Beer Store', 'Hotel', 'Health Food Store',
       'Antique Shop', 'Boutique', 'Furniture / Home Store',
       'Vietnamese Restaurant', 'Clothing Store', 'Accessories Store',
       "Women's Store", 'Gift Shop', 'Miscellaneous Shop',
       'Italian Restaurant', 'Creperie', 'Beer Bar',
       'Arts & Crafts Store', 'Burrito Place', 'Hobby Shop',
       'Sushi Restaurant', 'Diner', 'Fried Chicken Joint', 'Wings Joint',
       'Jap

In [81]:
toronto_venues.groupby('Neighborhood').count()
search_values=['Restaurant']
t_venues=toronto_venues[toronto_venues['Venue Category'].str.contains('|'.join(search_values))]

### One Hot Encoding

In [82]:
# one hot encoding
toronto_onehot = pd.get_dummies(t_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = t_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Afghan Restaurant,American Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Colombian Restaurant,...,Portuguese Restaurant,Ramen Restaurant,Restaurant,Seafood Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
5,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
28,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [83]:
toronto_onehot.shape

(493, 47)

### Grouping Data by Neighborhoods 

In [84]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Afghan Restaurant,American Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Colombian Restaurant,...,Portuguese Restaurant,Ramen Restaurant,Restaurant,Seafood Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000
1,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.333333,0.000000,0.333333,0.0,0.000000,0.0,0.000000,0.000000
2,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000
3,"Bedford Park, Lawrence Manor East",0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.100000,0.000000,0.100000,0.0,0.100000,0.0,0.000000,0.000000
4,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.181818,0.181818,0.000000,0.0,0.090909,0.0,0.090909,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,Victoria Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000
58,Westmount,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000
59,"Wexford, Maryvale",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000
60,Willowdale,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.272727,0.181818,0.000000,0.090909,0.0,0.000000,0.0,0.000000,0.090909


In [ ]:
neighborhoods_venues_sorted=neighborhoods_venues_sorted[neighborhoods_venues_sorted.apply(lambda r: r.str.contains('Indian Restaurant', case=False).any(), axis=1)] 
neighborhoods_venues_sorted.reset_index(inplace=True)

In [132]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bedford Park, Lawrence Manor East",Italian Restaurant,Comfort Food Restaurant,American Restaurant,Thai Restaurant,Greek Restaurant,Sushi Restaurant,Restaurant,Indian Restaurant,Dim Sum Restaurant,Filipino Restaurant
1,Central Bay Street,Italian Restaurant,Thai Restaurant,Japanese Restaurant,Middle Eastern Restaurant,Modern European Restaurant,French Restaurant,Vegetarian / Vegan Restaurant,Indian Restaurant,Korean Restaurant,Falafel Restaurant
2,Church and Wellesley,Japanese Restaurant,Sushi Restaurant,Restaurant,Mediterranean Restaurant,Afghan Restaurant,Caribbean Restaurant,Indian Restaurant,Italian Restaurant,Mexican Restaurant,Fast Food Restaurant
3,Davisville,Sushi Restaurant,Italian Restaurant,Thai Restaurant,Greek Restaurant,Seafood Restaurant,Restaurant,Indian Restaurant,Japanese Restaurant,Vietnamese Restaurant,Dim Sum Restaurant
4,"Dorset Park, Wexford Heights, Scarborough Town...",Indian Restaurant,Chinese Restaurant,Vietnamese Restaurant,Asian Restaurant,Belgian Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant
5,"Harbourfront East, Union Station, Toronto Islands",Restaurant,Italian Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant,Seafood Restaurant,Indian Restaurant,New American Restaurant,Chinese Restaurant,Japanese Restaurant,Vietnamese Restaurant
6,"St. James Town, Cabbagetown",Restaurant,Italian Restaurant,American Restaurant,Thai Restaurant,Taiwanese Restaurant,Indian Restaurant,Caribbean Restaurant,Chinese Restaurant,Japanese Restaurant,Vietnamese Restaurant
7,"The Annex, North Midtown, Yorkville",Vegetarian / Vegan Restaurant,American Restaurant,Indian Restaurant,Middle Eastern Restaurant,Doner Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
8,"The Danforth West, Riverdale",Greek Restaurant,Italian Restaurant,Restaurant,American Restaurant,Indian Restaurant,Caribbean Restaurant,Doner Restaurant,German Restaurant,French Restaurant,Filipino Restaurant
9,Thorncliffe Park,Indian Restaurant,Fast Food Restaurant,Restaurant,Middle Eastern Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,French Restaurant,Filipino Restaurant,Falafel Restaurant,Ethiopian Restaurant


In [133]:
toronto_grouped=toronto_grouped[toronto_grouped['Neighborhood'].isin(neighborhoods_venues_sorted['Neighborhood'])]

In [139]:
toronto_grouped.reset_index(inplace=True)
toronto_grouped.drop('index',axis=1,inplace=True)

,Neighborhood,Afghan Restaurant,American Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Colombian Restaurant,...,Portuguese Restaurant,Ramen Restaurant,Restaurant,Seafood Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,"Bedford Park, Lawrence Manor East",0.00,0.100000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.000000,0.100000,0.000000,0.100000,0.000000,0.100000,0.00,0.000000,0.00
1,Central Bay Street,0.00,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.055556,0.055556,0.000000,0.055556,0.000000,0.111111,0.00,0.055556,0.00
2,Church and Wellesley,0.04,0.040000,0.0,0.0,0.0,0.0,0.040000,0.000000,0.0,...,0.0,0.040000,0.120000,0.000000,0.200000,0.000000,0.040000,0.04,0.000000,0.00
3,Davisville,0.00,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.000000,0.100000,0.100000,0.200000,0.000000,0.100000,0.00,0.000000,0.00
4,"Dorset Park, Wexford Heights, Scarborough Town...",0.00,0.000000,0.0,0.0,0.0,0.0,0.000000,0.250000,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.25
5,"Harbourfront East, Union Station, Toronto Islands",0.00,0.000000,0.0,0.0,0.0,0.0,0.000000,0.076923,0.0,...,0.0,0.000000,0.230769,0.076923,0.076923,0.000000,0.000000,0.00,0.076923,0.00
6,"St. James Town, Cabbagetown",0.00,0.083333,0.0,0.0,0.0,0.0,0.083333,0.083333,0.0,...,0.0,0.000000,0.250000,0.000000,0.000000,0.083333,0.083333,0.00,0.000000,0.00
7,"The Annex, North Midtown, Yorkville",0.00,0.250000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.250000,0.00
8,"The Danforth West, Riverdale",0.00,0.058824,0.0,0.0,0.0,0.0,0.058824,0.000000,0.0,...,0.0,0.000000,0.117647,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00
9,Thorncliffe Park,0.00,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00


### Creating new dataframe and displaying the top 10 venues for each neighborhood

In [85]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [96]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant,Eastern European Restaurant
1,"Bathurst Manor, Wilson Heights, Downsview North",Sushi Restaurant,Restaurant,Middle Eastern Restaurant,Vietnamese Restaurant,Cuban Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant
2,Bayview Village,Japanese Restaurant,Chinese Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant
3,"Bedford Park, Lawrence Manor East",Italian Restaurant,Comfort Food Restaurant,American Restaurant,Thai Restaurant,Greek Restaurant,Sushi Restaurant,Restaurant,Indian Restaurant,Dim Sum Restaurant,Filipino Restaurant
4,Berczy Park,Restaurant,Seafood Restaurant,Comfort Food Restaurant,Greek Restaurant,Vegetarian / Vegan Restaurant,Japanese Restaurant,Eastern European Restaurant,French Restaurant,Thai Restaurant,Brazilian Restaurant
...,...,...,...,...,...,...,...,...,...,...,...
57,Victoria Village,French Restaurant,Portuguese Restaurant,Vietnamese Restaurant,Cuban Restaurant,German Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant,Eastern European Restaurant
58,Westmount,Chinese Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant
59,"Wexford, Maryvale",Middle Eastern Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant,Eastern European Restaurant
60,Willowdale,Ramen Restaurant,Restaurant,Vietnamese Restaurant,Fast Food Restaurant,Sushi Restaurant,Indonesian Restaurant,Japanese Restaurant,Middle Eastern Restaurant,Cuban Restaurant,Filipino Restaurant


### Modelling: CLustering using K-means

In [140]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 2, 0, 0, 3, 4, 1], dtype=int32)

In [141]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merge = df2

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merge = toronto_merge.merge(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merge.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Italian Restaurant,Thai Restaurant,Japanese Restaurant,Middle Eastern Restaurant,Modern European Restaurant,French Restaurant,Vegetarian / Vegan Restaurant,Indian Restaurant,Korean Restaurant,Falafel Restaurant
1,M4H,East York,Thorncliffe Park,43.705369,-79.349372,1,Indian Restaurant,Fast Food Restaurant,Restaurant,Middle Eastern Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,French Restaurant,Filipino Restaurant,Falafel Restaurant,Ethiopian Restaurant
2,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,0,Restaurant,Italian Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant,Seafood Restaurant,Indian Restaurant,New American Restaurant,Chinese Restaurant,Japanese Restaurant,Vietnamese Restaurant
3,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,4,Greek Restaurant,Italian Restaurant,Restaurant,American Restaurant,Indian Restaurant,Caribbean Restaurant,Doner Restaurant,German Restaurant,French Restaurant,Filipino Restaurant
4,M5M,North York,"Bedford Park, Lawrence Manor East",43.733283,-79.419750,0,Italian Restaurant,Comfort Food Restaurant,American Restaurant,Thai Restaurant,Greek Restaurant,Sushi Restaurant,Restaurant,Indian Restaurant,Dim Sum Restaurant,Filipino Restaurant


### Mapping Clusters

In [142]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merge['Latitude'], toronto_merge['Longitude'], toronto_merge['Neighborhood'], toronto_merge['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Thanks